In [1]:
import gym

In [2]:
env = gym.make('Breakout-v0')

### Random Agent

In [3]:
episodes = 100

for episode in range(1, episodes):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        state, reward, done, info=env.step(env.action_space.sample())
        score += reward
    print('Episode : {}\nScore: {}\nInfo: {}'.format(episode,score,info))
    
env.close()

Episode : 1
Score: 1.0
Info: {'ale.lives': 0}
Episode : 2
Score: 2.0
Info: {'ale.lives': 0}
Episode : 3
Score: 1.0
Info: {'ale.lives': 0}
Episode : 4
Score: 1.0
Info: {'ale.lives': 0}
Episode : 5
Score: 0.0
Info: {'ale.lives': 0}
Episode : 6
Score: 1.0
Info: {'ale.lives': 0}
Episode : 7
Score: 0.0
Info: {'ale.lives': 0}
Episode : 8
Score: 3.0
Info: {'ale.lives': 0}
Episode : 9
Score: 2.0
Info: {'ale.lives': 0}
Episode : 10
Score: 1.0
Info: {'ale.lives': 0}
Episode : 11
Score: 0.0
Info: {'ale.lives': 0}
Episode : 12
Score: 3.0
Info: {'ale.lives': 0}
Episode : 13
Score: 0.0
Info: {'ale.lives': 0}
Episode : 14
Score: 4.0
Info: {'ale.lives': 0}
Episode : 15
Score: 5.0
Info: {'ale.lives': 0}
Episode : 16
Score: 0.0
Info: {'ale.lives': 0}
Episode : 17
Score: 2.0
Info: {'ale.lives': 0}
Episode : 18
Score: 1.0
Info: {'ale.lives': 0}
Episode : 19
Score: 1.0
Info: {'ale.lives': 0}
Episode : 20
Score: 2.0
Info: {'ale.lives': 0}
Episode : 21
Score: 3.0
Info: {'ale.lives': 0}
Episode : 22
Score: 3.

### DQNA

In [4]:
#### Import NN Packs
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [5]:
def Bmodel(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model
    

In [6]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [7]:
model = Bmodel(height, width, channels, actions)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 11, 8, 64)      65600     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 4, 3, 64)       65600     
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1180160   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1

In [8]:
#### Importing Keras-rl2

from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [9]:
def Bagent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2 ,nb_steps=1000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                   nb_actions = actions, enable_dueling_network=True, 
                   nb_steps_warmup=1000)
    return dqn

In [10]:
dqn = Bagent(model, actions)

In [11]:
dqn.compile(Adam(learning_rate=0.001))

In [12]:
dqn.fit(env, nb_steps=1000, visualize=False ,verbose=1)

Training for 1000 steps ...
Interval 1 (0 steps performed)
    6/10000 [..............................] - ETA: 2:01 - reward: 0.0000e+00 

c:\users\ghaiyur\documents\rl-agents\sirl\env\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  997/10000 [=>............................] - ETA: 1:46 - reward: 0.0040done, took 11.983 seconds


In [ ]:
scores = dqn.test(env, nb_episodes=50, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 50 episodes ...


In [ ]:
dqn.save_weights('models/BOdqn.h5f')

In [ ]:
# dqn.load_weights('models/dqn.h5f')